In [1]:
import pandas as pd
import time
import json
import os
from selenium import webdriver
from selenium.webdriver.common.by import By
import requests
from bs4 import BeautifulSoup

#json 靜態爬
    #transcript_name =request.POST['transcript_id'] #由transcript.js回傳
transcript_name = input("Enter the transcript name :")

#Y40B10A.2a.1
#F54H12.1c.3
response = requests.get(
        "https://wormbase.org/rest/widget/transcript/" + transcript_name + "/sequences",
        headers={
            "User-Agent": "Mozilla/5.0 (X11; Linux x86_64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/114.0.0.0 Safari/537.36"
        })

soup = BeautifulSoup(response.text, "html.parser")
data= json.loads(str(soup))

start = []
stop = []
#處理變成dict
strand = data["fields"]['unspliced_sequence_context']['data']['strand']
#check + -
if strand == "+" :
    data1 = data["fields"]['unspliced_sequence_context']['data']['positive_strand']['sequence']
    data2 = data["fields"]['spliced_sequence_context']['data']['positive_strand']['sequence']
    for i in range(0,len(data2)-1):
        if (data2[i].islower() & data2[i+1].isupper()):
            start.append(i+2)
        elif (data2[i].isupper() & data2[i+1].islower()):
            stop.append(i+1)
    #到時候記得改平行化
    if (stop == []):
        stop.append(len(data2))
    if (start == []):
        start.append(1)
else:
    data1 = data["fields"]['unspliced_sequence_context']['data']['negative_strand']['sequence']
    data2 = data["fields"]['spliced_sequence_context']['data']['negative_strand']['sequence']
    for i in range(0,len(data2)-1):
        if (data2[i].islower() & data2[i+1].isupper()):
            start.append(i+2)
        elif (data2[i].isupper() & data2[i+1].islower()):
            stop.append(i+1)
    #到時候記得改平行化
    if (stop == []):
        stop.append(len(data2))
    if (start == []):
        start.append(1)

#靜態爬

#check + -
if strand == "+" :
    transcriptdata1 = data["fields"]['unspliced_sequence_context']['data']['positive_strand']['features']
    transcriptdata2 = data["fields"]['spliced_sequence_context']['data']['positive_strand']['features']
    
else:
    transcriptdata1= data["fields"]['unspliced_sequence_context']['data']['negative_strand']['features']
    transcriptdata2= data["fields"]['spliced_sequence_context']['data']['negative_strand']['features']

def count_len(start: int,end: int):
    return end-start +1

unspliced = pd.DataFrame(columns = ['type','start','stop'])
spliced = pd.DataFrame(columns = ['type','start','stop'])

for i in range(len(transcriptdata1)):
    unspliced.loc[i] = transcriptdata1[i]

for i in range(len(transcriptdata2)):
    spliced.loc[i] = transcriptdata2[i]


CDS = {
'type': 'CDS',
'start':start[len(start)-1],
'stop':stop[len(stop)-1]}

CDS = pd.DataFrame([CDS])
df1 = pd.DataFrame()
df1 = spliced
spliced = pd.concat([spliced, CDS], ignore_index=True)

# Identify the row with the specified name
move_3 = df1[df1['type'] == 'three_prime_UTR']
move_5 = df1[df1['type'] == 'five_prime_UTR']
# Remove the identified row from its current position
df1 = df1[df1['type'] != 'three_prime_UTR']
df1 = df1[df1['type'] != 'five_prime_UTR']
# Insert the row at the top of the DataFrame
df1 = pd.concat([move_3, df1], ignore_index=True)
df1 = pd.concat([move_5, df1], ignore_index=True)
#reset sort
specific_row_index = 1  # Replace with the desired index
# Insert CDS at the specified row position in spliced
df1 = pd.concat([df1.iloc[:specific_row_index], CDS, df1.iloc[specific_row_index:]], ignore_index=True)

spliced_sp = ['type','start','stop']       
spliced_sp = df1[spliced_sp]




unspliced['length'] = unspliced.apply(lambda x: count_len(x['start'],x['stop']),axis=1)
spliced['length'] = spliced.apply(lambda x: count_len(x['start'],x['stop']),axis=1)
spliced_sp['length'] = spliced_sp.apply(lambda x: count_len(x['start'],x['stop']),axis=1)
#重新命名
#unspliced 
count_Intron_u= 1
count_Exon_u = 1
for k in range(0,len(unspliced)):
    if unspliced.at[k,'type'] == 'intron':
        unspliced.at[k,'type'] = 'Intron' + str(count_Intron_u)
        count_Intron_u+= 1
    elif unspliced.at[k,'type'] == 'exon':
        unspliced.at[k,'type'] = 'Exon' + str(count_Exon_u)
        count_Exon_u += 1
    elif unspliced.at[k,'type'] == 'five_prime_UTR':
        unspliced.at[k,'type'] = '5\'UTR'
    elif unspliced.at[k,'type'] == 'three_prime_UTR':
        unspliced.at[k,'type'] = '3\'UTR'
#spliced
count_Exon_s = 1
for k in range(0,len(spliced)):
    if spliced.at[k,'type'] == 'exon':
        spliced.at[k,'type'] = 'Exon' + str(count_Exon_s)
        count_Exon_s += 1
    elif spliced.at[k,'type'] == 'five_prime_UTR':
        spliced.at[k,'type'] = '5\'UTR'
    elif spliced.at[k,'type'] == 'three_prime_UTR':
        spliced.at[k,'type'] = '3\'UTR'
#用return 才能使用function內的變數

    

In [2]:
print(spliced_sp)
print(spliced)

              type  start  stop  length
0   five_prime_UTR      1   131     131
1              CDS    132   812     681
2  three_prime_UTR    813   868      56
3             exon      1   191     191
4             exon    192   522     331
5             exon    523   656     134
6             exon    657   868     212
    type  start  stop  length
0  Exon1      1   191     191
1  5'UTR      1   131     131
2  Exon2    192   522     331
3  Exon3    523   656     134
4  Exon4    657   868     212
5  3'UTR    813   868      56
6    CDS    132   812     681


In [3]:
spliced = spliced.values.tolist()
spliced_d3 = []
for i in range(0,len(spliced)):
    if (spliced[i][0] == '5\'UTR'):
        spliced_d3.append({"Type":'5\'UTR'})
        index = len(spliced_d3) - 1 
        spliced_d3[index]["start"] = spliced[i][1]
        spliced_d3[index]["end"] = spliced[i][2]
        spliced_d3[index]["x1"] = spliced[i][1]-1+10
        spliced_d3[index]["y1"] = 65
        spliced_d3[index]["x2"] = spliced[i][2]+10
        spliced_d3[index]["y2"] = 65
        spliced_d3[index]["color"] = "gray"         
    elif(spliced[i][0] == '3\'UTR'):
        spliced_d3.append({"Type":'3\'UTR'})
        index = len(spliced_d3) - 1 
        spliced_d3[index]["start"] = spliced[i][1]
        spliced_d3[index]["end"] = spliced[i][2]
        spliced_d3[index]["x1"] = spliced[i][1]-1+10
        spliced_d3[index]["y1"] = 65
        spliced_d3[index]["x2"] = spliced[i][2]+10
        spliced_d3[index]["y2"] = 65
        spliced_d3[index]["color"] = "gray"
    elif(spliced[i][0] == 'CDS'):
        spliced_d3.append({"Type":'CDS'})
        index = len(spliced_d3) - 1 
        spliced_d3[index]["start"] = spliced[i][1]
        spliced_d3[index]["end"] = spliced[i][2]
        spliced_d3[index]["x1"] = spliced[i][1]-1+10
        spliced_d3[index]["y1"] = 65
        spliced_d3[index]["x2"] = spliced[i][2]+10
        spliced_d3[index]["y2"] = 65
        spliced_d3[index]["color"] = "blue"   
    else:
        continue

          

In [4]:
print(spliced_d3)

[{'Type': "5'UTR", 'start': 1, 'end': 131, 'x1': 10, 'y1': 65, 'x2': 141, 'y2': 65, 'color': 'gray'}, {'Type': "3'UTR", 'start': 813, 'end': 868, 'x1': 822, 'y1': 65, 'x2': 878, 'y2': 65, 'color': 'gray'}, {'Type': 'CDS', 'start': 132, 'end': 812, 'x1': 141, 'y1': 65, 'x2': 822, 'y2': 65, 'color': 'blue'}]


In [5]:
cover  = 1
count = 0
y = 0
y_list = [0]
first = 1
while cover != 0:    
    cover = 0
    start = 0
    end = 0
    first = 1
    #繪製Y的高度這邊比對方式是，如果有與同一層的前一項重疊，則高度往上加，但這樣判斷會有錯誤，因為可能與前一層有重疊，但沒與更低層重疊
    for i in range(0,len(table_dict)):
        if table_dict[i]['y'] == y:
            if first == 1:
                start = table_dict[i]['start']
                end = table_dict[i]['end']
                first = 0
            elif first != 1 :
                #先判斷目前的開始點是否有被包在前一點的start以及end內，判斷完後再將start、end設定成現在判斷的start、end，讓下一個判斷使用
                #若有判斷到，則y會加一層
                if (table_dict[i]['start'] > start) and (table_dict[i]['start'] < end):
                    table_dict[i]['y'] = y+2.1
                    cover = 1
                else: 
                    start = table_dict[i]['start']
                    end = table_dict[i]['end']   
                    pass
    #print('========================================')
    y += 2.1
    y_list.append(y)
y+=4.2
y_list_inverse = y_list[::-1]

NameError: name 'table_dict' is not defined